In [21]:
import os

import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
dic_compteur_fruit = {}
i=0
for root, dir, filenames in os.walk('fruits-360_dataset/fruits-360/Training'):
    for filename in filenames:
        
        name = root.split("\\")[-1]
        n = name.split(" ")[0]
        if n in list(dic_compteur_fruit.keys()):
            dic_compteur_fruit[n] += 1
        else:
            dic_compteur_fruit[n] = 0
        old_path = root + "/" + filename
        newpath = "/".join(root.split("/")[0:-1]) + "/train/" + n + "/" + n + "_" + str(dic_compteur_fruit[n])+".jpg"
        os.rename(old_path,newpath) 

In [3]:
dic_compteur_fruit = {}
i=0
for root, dir, filenames in os.walk('fruits-360_dataset/fruits-360/Test'):
    for filename in filenames:
        
        name = root.split("\\")[-1]
        n = name.split(" ")[0]
        if n in list(dic_compteur_fruit.keys()):
            dic_compteur_fruit[n] += 1
        else:
            dic_compteur_fruit[n] = 0
        old_path = root + "/" + filename
        newpath = "/".join(root.split("/")[0:-1]) + "/testing/" + n + "/" + n + "_" + str(dic_compteur_fruit[n])+".jpg"
        os.rename(old_path,newpath) 

In [4]:
dic_compteur_fruit

{}

In [5]:
train_gen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range = 0.2,
    horizontal_flip=True,
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    dtype=tf.float32
)

In [6]:
BATCH_SIZE = 256
IMG_SIZE = 75
TRAIN_DIR = 'fruits-360_dataset/fruits-360/train'
TEST_DIR = 'fruits-360_dataset/fruits-360/testing'

In [64]:
train_batch = train_gen.flow_from_directory(
    directory=TRAIN_DIR,
    target_size=(IMG_SIZE,IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    seed=1
)

Found 70420 images belonging to 78 classes.


In [65]:
test_batch = train_gen.flow_from_directory(
    directory=TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode = 'sparse',
    seed=1
)

Found 22993 images belonging to 78 classes.


In [9]:
IMG_SHAPE = (IMG_SIZE,IMG_SIZE) + (3,)
base_model = tf.keras.applications.ResNet50(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [69]:
base_model.trainable = False
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(78, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

In [70]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [83]:
h = model.fit(train_batch, 
            epochs=50,
            steps_per_epoch = 6000 // BATCH_SIZE,
            validation_data = test_batch,
            validation_steps = 3000 // BATCH_SIZE,
            callbacks=tf.keras.callbacks.EarlyStopping(patience=3)
            )

Epoch 1/50
 6/23 [======>.......................] - ETA: 20s - loss: 0.1744 - accuracy: 0.9583

c:\Users\boure\anaconda3\envs\aifruits\lib\site-packages\PIL\Image.py:976: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


23/23 [==============================] - 42s 2s/step - loss: 0.1689 - accuracy: 0.9540 - val_loss: 0.2182 - val_accuracy: 0.9379
Epoch 2/50
23/23 [==============================] - 41s 2s/step - loss: 0.1612 - accuracy: 0.9582 - val_loss: 0.2169 - val_accuracy: 0.9347
Epoch 3/50
23/23 [==============================] - 45s 2s/step - loss: 0.1574 - accuracy: 0.9589 - val_loss: 0.2119 - val_accuracy: 0.9389
Epoch 4/50
23/23 [==============================] - 43s 2s/step - loss: 0.1488 - accuracy: 0.9589 - val_loss: 0.2016 - val_accuracy: 0.9411
Epoch 5/50
23/23 [==============================] - 42s 2s/step - loss: 0.1550 - accuracy: 0.9591 - val_loss: 0.2269 - val_accuracy: 0.9343
Epoch 6/50
23/23 [==============================] - 42s 2s/step - loss: 0.1596 - accuracy: 0.9545 - val_loss: 0.1804 - val_accuracy: 0.9489
Epoch 7/50
23/23 [==============================] - 44s 2s/step - loss: 0.1445 - accuracy: 0.9604 - val_loss: 0.2185 - val_accuracy: 0.9396
Epoch 8/50
23/23 [=============

In [87]:
model.save("aifruits_model.h5")

c:\Users\boure\anaconda3\envs\aifruits\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


## Test with real image

In [74]:
predicting_img = ImageDataGenerator(
    preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
    dtype=tf.float32
)

In [75]:
REAL_TEST_DIR = 'fruits-360_dataset/fruits-360/real_images'

In [76]:
img_transform = predicting_img.flow_from_directory(
    directory=REAL_TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE)
)

Found 10 images belonging to 1 classes.


In [84]:
prediction = model.predict(img_transform)

In [80]:
list_labels = [
    "Apple", "Apricot","Avocado","Banana", "Beetroot","Blueberry","Cabbage",
    "Cactus","Cantaloupe","Caspicum","Carambula","Carrot","Cauliflower","Cherry",
    "Chestnut","Clementine","Cocos","Corn","Cucumber","Dates","Eggplant","Fig",
    "Garlic","Ginger","Granadilla","Grape","Grapefruit","Guava","Hazelnut",
    "Huckleberry","Jalapeno","Kaki","Kiwi","Kohlrabi","Kumquats","Lemon","Limes",
    "Lychee","Mandarine","Mango","Mangostan","Maracuja","Melon","Mulberry",
    "Nectarine","Nut","Onion","Orange","Papaya","Passion","Peach","Pear",
    "Peas","Pepino","Pepper","Physalis","Pineapple","Pitahaya","Plum","Pomegranate",
    "Pomelo","Potato","Quince","Quince","Raddish","Rambutan","Raspberry","Redcurrant",
    "Salak","Soybeans","Spinach","Strawberry","Tamarillo","Tangelo","Tomato","Turnip",
    "Walnut","Watermelon"
]

In [81]:
len(list_labels)

78

In [85]:
for item in prediction:
    liste=[]
    for pred in np.argsort(item)[-5:]:
        liste.append(list_labels[pred])
    print(liste)

['Watermelon', 'Mangostan', 'Apple', 'Physalis', 'Caspicum']
['Maracuja', 'Watermelon', 'Pomelo', 'Peas', 'Strawberry']
['Cherry', 'Mangostan', 'Caspicum', 'Tomato', 'Apple']
['Maracuja', 'Watermelon', 'Pomelo', 'Peas', 'Strawberry']
['Physalis', 'Cherry', 'Tamarillo', 'Tomato', 'Apple']
['Tomato', 'Jalapeno', 'Cherry', 'Pomelo', 'Orange']
['Orange', 'Peas', 'Apple', 'Garlic', 'Turnip']
['Apple', 'Pomegranate', 'Mangostan', 'Tomato', 'Pomelo']
['Pomegranate', 'Tomato', 'Apple', 'Pomelo', 'Cherry']
['Cherry', 'Physalis', 'Peas', 'Tomato', 'Apple']


In [86]:
for item in prediction:
    print(np.max(item))

0.2620113
0.87740034
0.625669
0.87740034
0.7607616
0.29217407
0.64259315
0.47031373
0.42827544
0.9722272


In [92]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 75, 75, 3)]       0         
_________________________________________________________________
resnet50 (Functional)        (None, 3, 3, 2048)        23587712  
_________________________________________________________________
global_average_pooling2d_2 ( (None, 2048)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               1049088   
_________________________________________________________________
dense_7 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 78)                4001

In [109]:
base_model.trainable = False
for i in range(len(base_model.layers)):
    if i>=len(base_model.layers)*0.75:
        base_model.layers[i].trainable = True

43

In [112]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [113]:
h = model.fit(train_batch, 
            epochs=50,
            steps_per_epoch = 6000 // BATCH_SIZE,
            validation_data = test_batch,
            validation_steps = 3000 // BATCH_SIZE,
            callbacks=tf.keras.callbacks.EarlyStopping(patience=3)
            )

Epoch 1/50
23/23 [==============================] - 43s 2s/step - loss: 0.1490 - accuracy: 0.9564 - val_loss: 0.2306 - val_accuracy: 0.9379
Epoch 2/50
12/23 [==============>...............] - ETA: 13s - loss: 0.1611 - accuracy: 0.9518

c:\Users\boure\anaconda3\envs\aifruits\lib\site-packages\PIL\Image.py:976: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


23/23 [==============================] - 40s 2s/step - loss: 0.1570 - accuracy: 0.9543 - val_loss: 0.1728 - val_accuracy: 0.9499
Epoch 3/50
23/23 [==============================] - 38s 2s/step - loss: 0.1448 - accuracy: 0.9613 - val_loss: 0.2028 - val_accuracy: 0.9425
Epoch 4/50
23/23 [==============================] - 39s 2s/step - loss: 0.1364 - accuracy: 0.9618 - val_loss: 0.1720 - val_accuracy: 0.9492
Epoch 5/50
23/23 [==============================] - 38s 2s/step - loss: 0.1196 - accuracy: 0.9660 - val_loss: 0.1723 - val_accuracy: 0.9517
Epoch 6/50
23/23 [==============================] - 39s 2s/step - loss: 0.1255 - accuracy: 0.9631 - val_loss: 0.1650 - val_accuracy: 0.9499
Epoch 7/50
23/23 [==============================] - 39s 2s/step - loss: 0.1244 - accuracy: 0.9650 - val_loss: 0.1781 - val_accuracy: 0.9510
Epoch 8/50
23/23 [==============================] - 39s 2s/step - loss: 0.1272 - accuracy: 0.9637 - val_loss: 0.2074 - val_accuracy: 0.9425
Epoch 9/50
23/23 [=============

In [114]:
prediction = model.predict(img_transform)

In [115]:
for item in prediction:
    liste=[]
    for pred in np.argsort(item)[-5:]:
        liste.append(list_labels[pred])
    print(liste)

['Mangostan', 'Soybeans', 'Cherry', 'Tomato', 'Apple']
['Cauliflower', 'Peas', 'Apple', 'Garlic', 'Turnip']
['Pomelo', 'Cherry', 'Tamarillo', 'Tomato', 'Apple']
['Beetroot', 'Mangostan', 'Tomato', 'Apple', 'Pomelo']
['Peas', 'Mangostan', 'Watermelon', 'Caspicum', 'Apple']
['Maracuja', 'Watermelon', 'Pomelo', 'Peas', 'Strawberry']
['Jalapeno', 'Tomato', 'Cherry', 'Apple', 'Pomelo']
['Maracuja', 'Watermelon', 'Pomelo', 'Peas', 'Strawberry']
['Orange', 'Apple', 'Jalapeno', 'Tomato', 'Pomelo']
['Cherry', 'Redcurrant', 'Peas', 'Pomelo', 'Apple']


In [117]:
model.save("aifruits_model.h5")

c:\Users\boure\anaconda3\envs\aifruits\lib\site-packages\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


In [118]:
base_model.trainable = False
count=0
for i in range(len(base_model.layers)):
    if i>=len(base_model.layers)*0.5:
        base_model.layers[i].trainable = True
        count+=1
count

87

In [119]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [120]:
h = model.fit(train_batch, 
            epochs=50,
            steps_per_epoch = 6000 // BATCH_SIZE,
            validation_data = test_batch,
            validation_steps = 3000 // BATCH_SIZE,
            callbacks=tf.keras.callbacks.EarlyStopping(patience=3)
            )

Epoch 1/50
23/23 [==============================] - ETA: 0s - loss: 0.1288 - accuracy: 0.9625

c:\Users\boure\anaconda3\envs\aifruits\lib\site-packages\PIL\Image.py:976: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


23/23 [==============================] - 42s 2s/step - loss: 0.1288 - accuracy: 0.9625 - val_loss: 0.1660 - val_accuracy: 0.9492
Epoch 2/50
23/23 [==============================] - 38s 2s/step - loss: 0.1219 - accuracy: 0.9662 - val_loss: 0.1523 - val_accuracy: 0.9545
Epoch 3/50
23/23 [==============================] - 38s 2s/step - loss: 0.1099 - accuracy: 0.9681 - val_loss: 0.1492 - val_accuracy: 0.9528
Epoch 4/50
23/23 [==============================] - 39s 2s/step - loss: 0.1104 - accuracy: 0.9676 - val_loss: 0.1541 - val_accuracy: 0.9503
Epoch 5/50
23/23 [==============================] - 38s 2s/step - loss: 0.1103 - accuracy: 0.9682 - val_loss: 0.1631 - val_accuracy: 0.9542
Epoch 6/50
23/23 [==============================] - 38s 2s/step - loss: 0.1084 - accuracy: 0.9721 - val_loss: 0.1678 - val_accuracy: 0.9517


In [121]:
prediction = model.predict(img_transform)

In [122]:
for item in prediction:
    liste=[]
    for pred in np.argsort(item)[-5:]:
        liste.append(list_labels[pred])
    print(liste)

['Ginger', 'Apple', 'Orange', 'Turnip', 'Garlic']
['Jalapeno', 'Pomelo', 'Tomato', 'Cherry', 'Apple']
['Tomato', 'Apple', 'Orange', 'Pomelo', 'Cherry']
['Beetroot', 'Tomato', 'Mangostan', 'Apple', 'Pomelo']
['Mangostan', 'Cherry', 'Tamarillo', 'Tomato', 'Apple']
['Caspicum', 'Soybeans', 'Cherry', 'Tomato', 'Apple']
['Raddish', 'Peas', 'Apple', 'Pomelo', 'Strawberry']
['Peas', 'Cherry', 'Physalis', 'Pomelo', 'Apple']
['Raddish', 'Peas', 'Apple', 'Pomelo', 'Strawberry']
['Physalis', 'Jalapeno', 'Mangostan', 'Caspicum', 'Apple']


In [124]:
model.save("aifruits_model.h5")

In [125]:
base_model.trainable = False
count=0
for i in range(len(base_model.layers)):
    if i>=len(base_model.layers)*0.25:
        base_model.layers[i].trainable = True
        count+=1
count

131

In [126]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [127]:
h = model.fit(train_batch, 
            epochs=50,
            steps_per_epoch = 6000 // BATCH_SIZE,
            validation_data = test_batch,
            validation_steps = 3000 // BATCH_SIZE,
            callbacks=tf.keras.callbacks.EarlyStopping(patience=3)
            )

Epoch 1/50
19/23 [=======================>......] - ETA: 4s - loss: 0.1121 - accuracy: 0.9700

c:\Users\boure\anaconda3\envs\aifruits\lib\site-packages\PIL\Image.py:976: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


23/23 [==============================] - 41s 2s/step - loss: 0.1112 - accuracy: 0.9704 - val_loss: 0.1768 - val_accuracy: 0.9485
Epoch 2/50
23/23 [==============================] - 38s 2s/step - loss: 0.0947 - accuracy: 0.9718 - val_loss: 0.1670 - val_accuracy: 0.9496
Epoch 3/50
23/23 [==============================] - 38s 2s/step - loss: 0.1084 - accuracy: 0.9682 - val_loss: 0.1446 - val_accuracy: 0.9624
Epoch 4/50
23/23 [==============================] - 41s 2s/step - loss: 0.0970 - accuracy: 0.9737 - val_loss: 0.1606 - val_accuracy: 0.9524
Epoch 5/50
23/23 [==============================] - 40s 2s/step - loss: 0.1016 - accuracy: 0.9706 - val_loss: 0.1508 - val_accuracy: 0.9531
Epoch 6/50
23/23 [==============================] - 40s 2s/step - loss: 0.1028 - accuracy: 0.9701 - val_loss: 0.1528 - val_accuracy: 0.9535


In [128]:
prediction = model.predict(img_transform)

In [130]:
for item in prediction:
    liste=[]
    for pred in np.argsort(item)[-5:]:
        liste.append(list_labels[pred])
    print(liste)

['Cherry', 'Jalapeno', 'Orange', 'Pomelo', 'Apple']
['Caspicum', 'Mangostan', 'Tomato', 'Cherry', 'Apple']
['Watermelon', 'Caspicum', 'Physalis', 'Mangostan', 'Apple']
['Raddish', 'Peas', 'Apple', 'Pomelo', 'Strawberry']
['Cherry', 'Tomato', 'Pomelo', 'Jalapeno', 'Apple']
['Tomato', 'Beetroot', 'Apple', 'Mangostan', 'Pomelo']
['Orange', 'Cauliflower', 'Turnip', 'Apple', 'Garlic']
['Raddish', 'Peas', 'Apple', 'Pomelo', 'Strawberry']
['Physalis', 'Cherry', 'Tomato', 'Tamarillo', 'Apple']
['Peas', 'Redcurrant', 'Physalis', 'Pomelo', 'Apple']


In [131]:
model.save("aifruits_model.h5")

In [132]:
base_model.trainable = False
count=0
for i in range(len(base_model.layers)):
    if i>=len(base_model.layers)*0.15:
        base_model.layers[i].trainable = True
        count+=1
count

148

In [133]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [134]:
h = model.fit(train_batch, 
            epochs=50,
            steps_per_epoch = 6000 // BATCH_SIZE,
            validation_data = test_batch,
            validation_steps = 3000 // BATCH_SIZE,
            callbacks=tf.keras.callbacks.EarlyStopping(patience=3)
            )

Epoch 1/50
23/23 [==============================] - 41s 2s/step - loss: 0.1065 - accuracy: 0.9706 - val_loss: 0.1560 - val_accuracy: 0.9560
Epoch 2/50
23/23 [==============================] - 38s 2s/step - loss: 0.1039 - accuracy: 0.9701 - val_loss: 0.1360 - val_accuracy: 0.9620
Epoch 3/50
23/23 [==============================] - 39s 2s/step - loss: 0.0967 - accuracy: 0.9724 - val_loss: 0.1396 - val_accuracy: 0.9581
Epoch 4/50
23/23 [==============================] - 39s 2s/step - loss: 0.0916 - accuracy: 0.9737 - val_loss: 0.1467 - val_accuracy: 0.9592
Epoch 5/50
23/23 [==============================] - 40s 2s/step - loss: 0.0910 - accuracy: 0.9759 - val_loss: 0.1310 - val_accuracy: 0.9656
Epoch 6/50
23/23 [==============================] - 39s 2s/step - loss: 0.0816 - accuracy: 0.9774 - val_loss: 0.1330 - val_accuracy: 0.9595
Epoch 7/50
23/23 [==============================] - 40s 2s/step - loss: 0.0886 - accuracy: 0.9738 - val_loss: 0.1475 - val_accuracy: 0.9624
Epoch 8/50
23/23 [==

In [135]:
prediction = model.predict(img_transform)

In [136]:
for item in prediction:
    liste=[]
    for pred in np.argsort(item)[-5:]:
        liste.append(list_labels[pred])
    print(liste)

['Beetroot', 'Apple', 'Tomato', 'Mangostan', 'Pomelo']
['Caspicum', 'Watermelon', 'Mangostan', 'Physalis', 'Apple']
['Orange', 'Caspicum', 'Mangostan', 'Tomato', 'Apple']
['Peas', 'Caspicum', 'Physalis', 'Pomelo', 'Apple']
['Jalapeno', 'Pomegranate', 'Tomato', 'Pomelo', 'Apple']
['Maracuja', 'Watermelon', 'Apple', 'Pomelo', 'Strawberry']
['Orange', 'Cherry', 'Tamarillo', 'Tomato', 'Apple']
['Pomegranate', 'Pomelo', 'Tomato', 'Orange', 'Apple']
['Maracuja', 'Watermelon', 'Apple', 'Pomelo', 'Strawberry']
['Cauliflower', 'Apple', 'Orange', 'Turnip', 'Garlic']


In [139]:
base_model.trainable = False
count=0
for i in range(len(base_model.layers)):
    if i>=len(base_model.layers)*0.05:
        base_model.layers[i].trainable = True
        count+=1
count
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy']
)

In [146]:
h = model.fit(train_batch, 
            epochs=50,
            steps_per_epoch = 6000 // BATCH_SIZE,
            validation_data = test_batch,
            validation_steps = 3000 // BATCH_SIZE,
            callbacks=tf.keras.callbacks.EarlyStopping(patience=3)
            )

Epoch 1/50


KeyboardInterrupt: 

In [141]:
prediction = model.predict(img_transform)

In [142]:
for item in prediction:
    liste=[]
    for pred in np.argsort(item)[-5:]:
        liste.append(list_labels[pred])
    print(liste)

['Mangostan', 'Peas', 'Apple', 'Pomelo', 'Strawberry']
['Redcurrant', 'Peas', 'Cherry', 'Pomelo', 'Apple']
['Apple', 'Cucumber', 'Physalis', 'Mangostan', 'Caspicum']
['Mangostan', 'Orange', 'Eggplant', 'Grape', 'Apple']
['Cherry', 'Watermelon', 'Pomelo', 'Peas', 'Apple']
['Mangostan', 'Peas', 'Apple', 'Pomelo', 'Strawberry']
['Peas', 'Nut', 'Apple', 'Mangostan', 'Pomelo']
['Papaya', 'Cauliflower', 'Peas', 'Turnip', 'Garlic']
['Tomato', 'Apple', 'Pomelo', 'Cherry', 'Orange']
['Orange', 'Pomegranate', 'Mangostan', 'Apple', 'Tomato']


In [143]:
model = tf.keras.models.load_model("aifruits_model.h5")

In [144]:
prediction = model.predict(img_transform)

In [145]:
for item in prediction:
    liste=[]
    for pred in np.argsort(item)[-5:]:
        liste.append(list_labels[pred])
    print(liste)

['Caspicum', 'Mangostan', 'Tomato', 'Cherry', 'Apple']
['Raddish', 'Peas', 'Apple', 'Pomelo', 'Strawberry']
['Cherry', 'Tomato', 'Pomelo', 'Jalapeno', 'Apple']
['Tomato', 'Beetroot', 'Apple', 'Mangostan', 'Pomelo']
['Peas', 'Redcurrant', 'Physalis', 'Pomelo', 'Apple']
['Cherry', 'Jalapeno', 'Orange', 'Pomelo', 'Apple']
['Physalis', 'Cherry', 'Tomato', 'Tamarillo', 'Apple']
['Orange', 'Cauliflower', 'Turnip', 'Apple', 'Garlic']
['Raddish', 'Peas', 'Apple', 'Pomelo', 'Strawberry']
['Watermelon', 'Caspicum', 'Physalis', 'Mangostan', 'Apple']
